In [1]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

## Original Dataset

In [4]:
raw_data = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv',
                      skiprows=[0,1,2,3])
raw_data.columns =['city',
                   'population',
                   'violent_crime',
                   'murder',
                   'rape_def_1',
                   'rape_def_2',
                  'robbery',
                  'aggravated_assault',
                  'property_crime',
                  'burglary',
                  'larceny_theft',
                  'motor_vehicle_theft',
                  'arson']

#set dataframe
df = raw_data[['city', 'population']].copy()

#get population^2 variable
df = df.dropna(axis=0, subset=['population']) #drop nan values
df['population'] = df['population'].str.replace(',', '') #get rid of commas
df['population'] = df['population'].astype(str).astype(int) #from object cast to string then int
df['population_sq'] = df['population'] ** 2 #square column and put in new column

#add other desired variables
df['murder'], df['robbery'] = raw_data['murder'], raw_data['robbery']
df.head()

#prepare murder and robbery to be iterated on
df['murder'] = df['murder'].astype(int)
df['robbery'] = df['robbery'].str.replace(',', '')
df['robbery'] = df['robbery'].astype(int)

#change values to 0 or 1 for multivariable regression
df.loc[df['murder'] > 0, ['murder']] = 1
df.loc[df['robbery'] > 0, ['robbery']] = 1

#add property crime variable
df['property_crime'] = raw_data['property_crime']
df['property_crime'] = df['property_crime'].str.replace(',', '') #get rid of commas
df['property_crime'] = df['property_crime'].astype(int)

In [10]:
#use regression that eliminated the population feature
df_regr = df[['population_sq','murder','robbery','property_crime']].copy()

regr = linear_model.LinearRegression()
Y = df_regr['property_crime'].values.reshape(-1,1)
X = df_regr[['population_sq','murder','robbery']]
regr.fit(X,Y)

#inspect results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

correlation_matrix = X.corr()
display(correlation_matrix)


Coefficients: 
 [[1.99256120e-09 1.18406016e+03 3.00052598e+02]]

Intercept: 
 [40.08610471]

R-squared:
0.9859871204030131


,population_sq,murder,robbery
population_sq,1.000,0.133,0.044
murder,0.133,1.000,0.313
robbery,0.044,0.313,1.000


## Test on new datasets

In [21]:
#crime data for Illinois 2013
IL_2013_raw = pd.read_excel('table_8_offenses_known_to_law_enforcement_illinois_by_city_2013.xls',
                       skiprows=[0,1,2,3])
IL_2013_raw.columns =['city',
                   'population',
                   'violent_crime',
                   'murder',
                   'rape_def_1',
                   'rape_def_2',
                  'robbery',
                  'aggravated_assault',
                  'property_crime',
                  'burglary',
                  'larceny_theft',
                  'motor_vehicle_theft',
                  'arson']

IL_2013_raw = pd.DataFrame(IL_2013_raw)
IL_2013_raw.head()

,city,population,violent_crime,murder,rape_def_1,rape_def_2,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson
0,Addison,37378.000,40.000,1.000,nan,8.000,5.000,26.000,640.000,97.000,527.000,16.000,5.000
1,Albany,878.000,0.000,0.000,nan,0.000,0.000,0.000,5.000,0.000,5.000,0.000,0.000
2,Albers,1187.000,0.000,0.000,nan,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,Albion,1975.000,0.000,0.000,nan,0.000,0.000,0.000,33.000,8.000,22.000,3.000,1.000
4,Algonquin,29980.000,18.000,0.000,nan,3.000,2.000,13.000,519.000,33.000,475.000,11.000,6.000


In [32]:
#prepare dataframe for regression
IL_2013 = IL_2013_raw[['city', 'population']].copy()

#get population^2 variable
IL_2013 = IL_2013.dropna(axis=0, subset=['population']) #drop nan values
IL_2013['population_sq'] = IL_2013['population'] ** 2 #square column and put in new column

#add other desired variables
IL_2013['murder'], IL_2013['robbery'] = IL_2013_raw['murder'], IL_2013_raw['robbery']

#prepare murder and robbery to be iterated on
IL_2013['murder'] = IL_2013['murder'].astype(int)
IL_2013['robbery'] = IL_2013['robbery'].astype(int)

#change values to 0 or 1 for multivariable regression
IL_2013.loc[IL_2013['murder'] > 0, ['murder']] = 1
IL_2013.loc[IL_2013['robbery'] > 0, ['robbery']] = 1

#add property crime variable
IL_2013['property_crime'] = IL_2013_raw['property_crime']
IL_2013['property_crime'] = IL_2013['property_crime'].astype(int)

IL_2013.head()

,city,population,population_sq,murder,robbery,property_crime
0,Addison,37378.000,1397114884.000,1,1,640
1,Albany,878.000,770884.000,0,0,5
2,Albers,1187.000,1408969.000,0,0,0
3,Albion,1975.000,3900625.000,0,0,33
4,Algonquin,29980.000,898800400.000,0,1,519


In [31]:
#run model
IL_regr = IL_2013[['population_sq','murder','robbery','property_crime']].copy()

Y_IL = IL_regr['property_crime'].values.reshape(-1,1)
X_IL = IL_regr[['population_sq','murder','robbery']]
regr.fit(X_IL,Y_IL)

#inspect results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_IL, Y_IL))

correlation_matrix = X_IL.corr()
display(correlation_matrix)


Coefficients: 
 [[1.28506961e-08 5.03162440e+02 3.27592903e+02]]

Intercept: 
 [27.51233757]

R-squared:
0.9861143287517039


,population_sq,murder,robbery
population_sq,1.000,0.099,0.042
murder,0.099,1.000,0.309
robbery,0.042,0.309,1.000


In [34]:
#crime data for New York 2014
NY_2014_raw = pd.read_excel('Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.xls',
                       skiprows=[0,1,2,3])
NY_2014_raw.columns =['city',
                   'population',
                   'violent_crime',
                   'murder',
                   'rape_def_1',
                   'rape_def_2',
                  'robbery',
                  'aggravated_assault',
                  'property_crime',
                  'burglary',
                  'larceny_theft',
                  'motor_vehicle_theft',
                  'arson']

NY_2014_raw = pd.DataFrame(NY_2014_raw)
NY_2014_raw.head()

,city,population,violent_crime,murder,rape_def_1,rape_def_2,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson
0,Adams Village,1851.000,0.000,0.000,nan,0.000,0.000,0.000,11.000,1.000,10.000,0.000,0.000
1,Addison Town and Village,2568.000,2.000,0.000,nan,0.000,1.000,1.000,49.000,1.000,47.000,1.000,0.000
2,Afton Village4,820.000,0.000,0.000,0.000,nan,0.000,0.000,1.000,0.000,1.000,0.000,0.000
3,Akron Village,2842.000,1.000,0.000,nan,0.000,0.000,1.000,17.000,0.000,17.000,0.000,0.000
4,Albany4,98595.000,802.000,8.000,54.000,nan,237.000,503.000,3888.000,683.000,3083.000,122.000,12.000


In [39]:
#prepare dataframe for regression
NY_2014 = NY_2014_raw[['city', 'population']].copy()

#get population^2 variable
NY_2014 = NY_2014.dropna(axis=0, subset=['population']) #drop nan values
NY_2014['population_sq'] = NY_2014['population'] ** 2 #square column and put in new column

#add other desired variables
NY_2014['murder'], NY_2014['robbery'] = NY_2014_raw['murder'], NY_2014_raw['robbery']

#prepare murder and robbery to be iterated on
NY_2014['murder'] = NY_2014['murder'].astype(int)
NY_2014['robbery'] = NY_2014['robbery'].astype(int)

#change values to 0 or 1 for multivariable regression
NY_2014.loc[NY_2014['murder'] > 0, ['murder']] = 1
NY_2014.loc[NY_2014['robbery'] > 0, ['robbery']] = 1

#add property crime variable
NY_2014['property_crime'] = NY_2014_raw['property_crime']
NY_2014 = NY_2014.dropna(axis=0, subset=['property_crime'])
NY_2014['property_crime'] = NY_2014['property_crime'].astype(int)

NY_2014.head()

,city,population,population_sq,murder,robbery,property_crime
0,Adams Village,1851.000,3426201.000,0,0,11
1,Addison Town and Village,2568.000,6594624.000,0,1,49
2,Afton Village4,820.000,672400.000,0,0,1
3,Akron Village,2842.000,8076964.000,0,0,17
4,Albany4,98595.000,9720974025.000,1,1,3888


In [40]:
#run model
NY_2014_regr = NY_2014[['population_sq','murder','robbery','property_crime']].copy()

Y_NY_2014 = NY_2014_regr['property_crime'].values.reshape(-1,1)
X_NY_2014 = NY_2014_regr[['population_sq','murder','robbery']]
regr.fit(X_NY_2014,Y_NY_2014)

#inspect results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_NY_2014, Y_NY_2014))

correlation_matrix = X_NY_2014.corr()
display(correlation_matrix)


Coefficients: 
 [[1.86957569e-09 1.21568897e+03 2.87083640e+02]]

Intercept: 
 [11.22027883]

R-squared:
0.9868393267774052


,population_sq,murder,robbery
population_sq,1.000,0.131,0.049
murder,0.131,1.000,0.315
robbery,0.049,0.315,1.000
